# This notebook contains Step 2 : Answer Generation and  Step 3 Assertions part of the pipeline

In [1]:
import json
import os
import numpy as np
import pandas as pd
import json
import re
import pandas as pd
from tqdm import tqdm  
from sentence_transformers import CrossEncoder
from transformers import pipeline

2025-09-26 10:26:50.753258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758900410.772347 3999145 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758900410.778159 3999145 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-26 10:26:50.805590: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import os
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import warnings
from transformers.utils import logging as hf_logging

In [3]:
import warnings

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [4]:
hf_logging.set_verbosity_error()

warnings.filterwarnings("ignore", category=FutureWarning)

### Fetch data from the ChromaDb Store

In [5]:
chroma_client = chromadb.PersistentClient(path="./ChromaDb")
collection_name = "qasper_data_collection"

chroma_collection = chroma_client.get_collection(collection_name)


In [6]:
questions_qasper = pd.read_csv('processed_qasper_data.csv')

In [7]:
questions_qasper.head()

,paper_id,title,abstract,full_text,question_id,question,nlp_background,topic_background,paper_read,search_query,question_writer,annotation_id,worker_id,unanswerable,yes_no,free_form_answer,extractive_spans,evidence,highlighted_evidence
0,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,753990d0b621d390ed58f20c4d9e4f065f0dc672,What is the seed lexicon?,two,unfamiliar,no,NaN,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,31e85022a847f37c15fd0415f3c450c74c8e4755,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,False,NaN,a vocabulary of positive and negative predicat...,NaN,The seed lexicon consists of positive and nega...,The seed lexicon consists of positive and nega...
1,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,9d578ddccc27dd849244d632dd0f6bf27348ad81,What are the results?,two,unfamiliar,no,NaN,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,1e5e867244ea656c4b7632628086209cf9bae5fa,2cfd959e433f290bb50b55722370f0d22fe090b7,False,NaN,Using all data to train: AL -- BiGRU achieved ...,NaN,FLOAT SELECTED: Table 3: Performance of variou...,FLOAT SELECTED: Table 3: Performance of variou...
2,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,02e4bf719b1a504e385c35c6186742e720bcb281,How are relations used to propagate polarity?,two,unfamiliar,no,NaN,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,49a78a07d2eed545556a835ccf2eb40e5eee9801,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,False,NaN,"based on the relation between events, the sugg...",NaN,"In this paper, we propose a simple and effecti...","As illustrated in Figure FIGREF1, our key idea..."
3,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,02e4bf719b1a504e385c35c6186742e720bcb281,How are relations used to propagate polarity?,two,unfamiliar,no,NaN,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,acd6d15bd67f4b1496ee8af1c93c33e7d59c89e1,2cfd959e433f290bb50b55722370f0d22fe090b7,False,NaN,cause relation: both events in the relation sh...,NaN,"In this paper, we propose a simple and effecti...","As illustrated in Figure FIGREF1, our key idea..."
4,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,44c4bd6decc86f1091b5fc0728873d9324cdde4e,How big is the Japanese data?,two,unfamiliar,no,NaN,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,36926a4c9e14352c91111150aa4c6edcc5c0770f,2cfd959e433f290bb50b55722370f0d22fe090b7,False,NaN,7000000 pairs of events were extracted from th...,NaN,"As a raw corpus, we used a Japanese web corpus...","As a raw corpus, we used a Japanese web corpus..."


In [8]:
# keep only rows with a question
questions = (
    questions_qasper.groupby("question_id")
      .first()                                     
      .reset_index()[["question_id","question",
                      "paper_id","free_form_answer"]]
)


In [9]:
questions.head()

,question_id,question,paper_id,free_form_answer
0,0038b073b7cca847033177024f9719c971692042,How is the input triple translated to a slot-f...,1706.04115,"The relation R(x,y) is mapped onto a question ..."
1,00bcdffff7e055f99aaf1b05cf41c98e2748e948,What is the baseline method for the task?,1909.02764,For the emotion recognition from text they use...
2,00ef9cc1d1d60f875969094bb246be529373cb1d,What methodology is used to compensate for lim...,1904.07342,Influential tweeters ( who they define as indi...
3,01123a39574bdc4684aafa59c52d956b532d2e53,By how much does their method outperform state...,1905.10247,"AE-HCN outperforms by 17%, AE-HCN-CNN outperfo..."
4,01dc6893fc2f49b732449dfe1907505e747440b0,What debate topics are included in the dataset?,1906.03538,"Ethics, Gender, Human rights, Sports, Freedom ..."


In [10]:
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


In [11]:
def retrieve_chunks(question, paper_id=None, k=10):
    where_clause = {"paper_id": str(paper_id)} if paper_id else None

    result = chroma_collection.query(
        query_texts=[question],
        n_results=k,
        where=where_clause
    )
    
    pairs = [(question, doc) for doc in result["documents"][0]]
    scores = cross_encoder.predict(pairs)
    
    # sort by score descending
    reranked = [doc for _, doc in sorted(zip(scores, result["documents"][0]), key=lambda x: x[0], reverse=True)]
    final_chunks = reranked[:3]

    context_text = " ".join(final_chunks)

    
    context_text = " ".join(result["documents"][0])  # combine list of strings into one
    clean_chunk = re.sub(r"\$.*?\$", "", context_text)   # remove inline math
 
    # Return list of text chunks
    return clean_chunk


In [12]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer, util
import pandas as pd

QA_MODEL = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
qa_tokenizer = AutoTokenizer.from_pretrained(QA_MODEL)
qa_model     = AutoModelForQuestionAnswering.from_pretrained(QA_MODEL)

embedder = SentenceTransformer("all-MiniLM-L6-v2") 


In [17]:
def qa_with_confidence(question, context):
    inputs = qa_tokenizer(
        question,
        context,
        return_tensors="pt",
        max_length=512,
        truncation=True
    )
    with torch.no_grad():
        outputs = qa_model(**inputs)

    start_idx = outputs.start_logits.argmax()
    end_idx   = outputs.end_logits.argmax()

    answer_tokens = inputs.input_ids[0, start_idx:end_idx+1]
    decoded_answer = qa_tokenizer.decode(answer_tokens, skip_special_tokens=True)

    start_probs = F.softmax(outputs.start_logits, dim=-1)
    end_probs   = F.softmax(outputs.end_logits, dim=-1)
    confidence  = (start_probs[0, start_idx] * end_probs[0, end_idx]).item()

    return decoded_answer, confidence


In [18]:
def cosine_similarity(prediction, ground_truth):
    emb_pred = embedder.encode(prediction, convert_to_tensor=True)
    emb_gt   = embedder.encode(ground_truth, convert_to_tensor=True)
    return util.cos_sim(emb_pred, emb_gt).item() 

In [19]:
results = []
for _, row in questions.iterrows():
    chunks = retrieve_chunks(row.question, row.paper_id)
    
    if not chunks or chunks.strip() == "":
        print(f"Skipping {row.question_id} (no context retrieved)")
        continue
    
    pred, conf = qa_with_confidence(row.question, chunks)
    cos        = cosine_similarity(pred, str(row.free_form_answer))
    results.append({
        "question_id": row.question_id,
        "question" : row.question,
        "model_answer": pred,
        "ground_truth": str(row.free_form_answer),
        "confidence": conf,
        "cosine_sim": cos
    })

scores_df = pd.DataFrame(results)

print("Average Confidence :", scores_df["confidence"].mean())
print("Average Cosine Sim :", scores_df["cosine_sim"].mean())


Skipping 01edeca7b902ae3fd66264366bf548acea1db364 (no context retrieved)
Skipping 0b54032508c96ff3320c3db613aeb25d42d00490 (no context retrieved)
Skipping 0bd683c51a87a110b68b377e9a06f0a3e12c8da0 (no context retrieved)
Skipping 11dde2be9a69a025f2fc29ce647201fb5a4df580 (no context retrieved)
Skipping 144714fe0d5a2bb7e21a7bf50df39d790ff12916 (no context retrieved)
Skipping 18fbf9c08075e3b696237d22473c463237d153f5 (no context retrieved)
Skipping 255fb6e20b95092c548ba47d8a295468e06698bd (no context retrieved)
Skipping 25e4dbc7e211a1ebe02ee8dff675b846fb18fdc5 (no context retrieved)
Skipping 271019168ed3a2b0ef5e3780b48a1ebefc562b57 (no context retrieved)
Skipping 2a1e6a69e06da2328fc73016ee057378821e0754 (no context retrieved)
Skipping 2d3bf170c1647c5a95abae50ee3ef3b404230ce4 (no context retrieved)
Skipping 2d47cdf2c1e0c64c73518aead1b94e0ee594b7a5 (no context retrieved)
Skipping 2fa0b9d0cb26e1be8eae7e782ada6820bc2c037f (no context retrieved)
Skipping 37753fbffc06ce7de6ada80c89f1bf5f190bbd88 (

In [ ]:
scores_df

In [38]:
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# print("Starting download and caching of Hugging Face models...")

# QA_MODELS = [
#       "deepset/tinyroberta-squad2",
#       "deepset/roberta-base-squad2",
#       "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
# ]

# for model_name in QA_MODELS:
#     try:
#         # print(f"Loading model: {model_name}")
#         # Load tokenizer
#         tokenizer = AutoTokenizer.from_pretrained(model_name)
        
#         # This is the essential part for downloading and caching the model
#         model = AutoModelForQuestionAnswering.from_pretrained(model_name)
        
#         # The following lines simulate a quick inference to ensure everything is loaded correctly
#         question = "How many programming languages does BLOOM support?"
#         context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
#         inputs = tokenizer(question, context, return_tensors="pt")

#         with torch.no_grad():
#             outputs = model(**inputs)

#         answer_start_index = outputs.start_logits.argmax()
#         answer_end_index = outputs.end_logits.argmax()
#         predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
#         decoded_answer = tokenizer.decode(predict_answer_tokens)
        
#         print(f"Successfully loaded and tested: {model_name}. Predicted answer: '{decoded_answer}'")

#     except Exception as e:
#         print(f"Failed to load model {model_name}. Error: {e}")

# print("Model loading process complete.")
     